In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set the path to the folder containing your CSV files
folder_path = R"C:\Users\andre\OneDrive - Alma Mater Studiorum Università di Bologna\University\UniBo\Machine Learning\PR2.20\data"

# List all files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv") if file != 'InfoComune.csv']

# Create an empty dictionary to store DataFrames
dataframes = {}

# Iterate through each CSV file
for file in csv_files:
    # Extract the file name (excluding .csv)
    df_name = os.path.splitext(file)[0]
    
    # Create the DataFrame and store it in the dictionary
    dataframes[df_name] = pd.read_csv(os.path.join(folder_path, file), header=0, skiprows = [1])


In [ ]:
for df in dataframes.values():
    # change missing values to the proper format
    df.replace('---', np.nan, inplace = True)
    # ensure a unique format
    df = df.convert_dtypes()

In [ ]:
dataframes['bologna'].columns.values

In [ ]:
dataframes['bologna']

In [ ]:
# columns to keep the average value only
pollutants = ['CO', 'NH3', 'NMVOC', 'NO2', 'NO', 'O3', 'PANS', 'PM10', 'PM2.5', 'SO2']

# metereological information
met = ['TG', 'TN', 'TX', 'HU', 'PP', 'QQ', 'RR']
met_pos = range(6, 13)
# date values
date = ['YYYY', 'MM', 'DD']
date_pos = list(range(3))

# rename the columns for date and metereological information
for df in dataframes.values():
    old_date = df.columns[date_pos]
    old_met = df.columns[met_pos]
    
    df.rename(columns=dict(zip(old_date, date)), inplace=True)
    df.rename(columns=dict(zip(old_met, met)), inplace=True)

In [ ]:
dataframes['bologna']

In [ ]:
selected_columns = date + met + pollutants

for province, df in dataframes.items():
    # Keep only selected columns
    dataframes[province] = df[selected_columns]

In [ ]:
dataframes['bologna']

In [ ]:
# create a date variable for all the dataframes
for province, df in dataframes.items():
    # Combine 'YYYY', 'MM', 'DD' columns into a new 'date' column
    df['date'] = pd.to_datetime(df[['YYYY', 'MM', 'DD']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')
    
    # Remove 'YYYY', 'MM', 'DD' columns
    df.drop(['YYYY', 'MM', 'DD'], axis=1, inplace=True)
    
    # Reorder columns with 'date' as the first column
    dataframes[province] = df[['date'] + [col for col in df.columns if col != 'date']]

In [ ]:
dataframes['bologna']

In [ ]:
numerics = met + pollutants

for province, df in dataframes.items():
    # first convert to numeric the columns in met and pollutants, since they are strings
    df[numerics] = df[numerics].apply(pd.to_numeric, errors = 'coerce')
    # round to the second decimal number for better visualization
    df[numerics] = df[numerics].round(2)

In [ ]:
dataframes['bologna']

In [ ]:
# we wanna filter the series so that we don't have missing values
# We'll start from 2018-01-01 and move until 2020-12-28

for province, df in dataframes.items():
    df = df[(df['date'] >= pd.to_datetime('2018-01-01')) & (df['date'] <= pd.to_datetime('2020-12-28'))]

    dataframes[province] = df

In [ ]:
dataframes['bologna']

In [ ]:
# we still have 1 missing value
dataframes['bologna'].isnull().sum()

In [ ]:
# the missing value is the 29th of February
dataframes['bologna'][dataframes['bologna'].isnull().any(axis=1)]

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

for province, df in dataframes.items():
    df[numerics] = imputer.fit_transform(df[numerics])
    
    dataframes[province] = df


In [ ]:
dataframes['bologna']

In [ ]:
dataframes['bologna'].isnull().sum()